https://learn.microsoft.com/en-us/rest/api/fabric/articles/

In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException, WorkspaceNotFoundException
import pandas as pd

client = fabric.FabricRestClient() # https://api.fabric.microsoft.com

In [ ]:
workspaceId = fabric.get_workspace_id()

response = client.get(f"/v1/workspaces/{workspaceId}/items")
df_items = pd.json_normalize(response.json()['value'])
df_items

In [ ]:
response = client.get(f"/v1/admin/workspaces")

In [ ]:
def obtener_elementos_area_trabajo(area_trabajo=None):
    """
    Obtiene los elementos de un área de trabajo específica de Microsoft Fabric.

    Args:
        area_trabajo (str, optional): El nombre o ID del área de trabajo. 
                                     Si es None, se utiliza el área de trabajo actual. 
                                     Por defecto es None.
    Returns:
        pandas.DataFrame: Un DataFrame con los elementos del área de trabajo, 
                          o None si ocurre un error o no se encuentran elementos.
    """
    df = None
    id_area_trabajo = None

    try:
        if area_trabajo is None:
            # Obtener el ID del área de trabajo actual si no se proporciona una específica
            id_area_trabajo = fabric.get_workspace_id()
        else:
            # Resolver el ID del área de trabajo a partir del nombre o ID proporcionado
            id_area_trabajo = fabric.resolve_workspace_id(area_trabajo)

        # Si se obtuvo un id_area_trabajo, proceder a obtener los elementos
        if id_area_trabajo:
            respuesta = client.get(f"/v1/workspaces/{id_area_trabajo}/items")

            # Verificar el código de estado de la respuesta
            if respuesta.status_code == 200:
                datos_respuesta = respuesta.json()
                df = pd.json_normalize(datos_respuesta['value'])
            else:
                print(f"Error en la respuesta de la API. Código de estado: {respuesta.status_code}, Respuesta: {respuesta.text}")
                raise FabricHTTPException(respuesta)

    except WorkspaceNotFoundException as e:
        print(f"Se detectó una WorkspaceNotFoundException: No se pudo encontrar el área de trabajo '{area_trabajo}'. Detalles: {e}")
    except FabricHTTPException as e:
        print(f"Se detectó una FabricHTTPException durante la operación. Verifique la configuración y los permisos. Detalles: {e}")
            
    return df

In [ ]:
obtener_elementos_area_trabajo()

In [ ]:
def crear_lakehouse(payload: dict, area_trabajo: str = None):
    """
    Crea un Lakehouse en Microsoft Fabric utilizando una solicitud POST.

    Args:
        payload (dict): El cuerpo de la solicitud para la creación del Lakehouse.
                        Debe contener al menos 'displayName' y 'type'.
        area_trabajo (str, optional): El nombre o ID del área de trabajo donde se creará el Lakehouse.
                                     Si es None, se utiliza el área de trabajo actual. 
                                     Por defecto es None.

    Returns:
        dict or None: La respuesta JSON del servidor si la creación es exitosa (código 201), 
                      o None si ocurre un error.
    """
    id_area_trabajo = None
    resultado = None

    try:
        if area_trabajo is None:
            id_area_trabajo = fabric.get_workspace_id()
        else:
            id_area_trabajo = fabric.resolve_workspace_id(area_trabajo)

        # Verificar que el payload contiene los campos mínimos requeridos
        if not all(k in payload for k in ("displayName", "type")):
            print("Error: El payload debe contener 'displayName' y 'type'.")
            return None
        
        if payload.get("type") != "Lakehouse":
            print(f"Advertencia: El tipo especificado en el payload es '{payload.get('type')}', se esperaba 'Lakehouse'.")

        print(f"Intentando crear el item '{payload.get('displayName')}' de tipo '{payload.get('type')}' en el área de trabajo ID: {id_area_trabajo}.")
        
        # Realizar la solicitud POST para crear el item (Lakehouse)
        respuesta = client.post(f"/v1/workspaces/{id_area_trabajo}/items", json=payload)

        # El código de estado esperado para una creación exitosa es 201 (Created)
        if respuesta.status_code == 201:
            print(f"El Item '{respuesta.json().get('displayName', payload.get('displayName'))}' ha sido creado correctamente.")
            resultado = respuesta.json() # Devolver la respuesta del servidor
        else:
            error_detalle = ""
            try:
                error_detalle = respuesta.json() 
            except ValueError:
                error_detalle = respuesta.text
            print(f"Error en la respuesta de la API al crear el item. Código de estado: {respuesta.status_code}. Detalles: {error_detalle}")
            raise FabricHTTPException(respuesta)

    except WorkspaceNotFoundException as e:
        print(f"Error: No se pudo encontrar el área de trabajo '{area_trabajo}'. Detalles: {e}")
    except FabricHTTPException as e:
        print(f"Error HTTP de Fabric durante la operación. Verifique el punto final de la API, la autenticación y el payload. Detalles: {e}")
    except Exception as e:
        identificador_ws = area_trabajo if area_trabajo else "actual"
        print(f"Ocurrió un error inesperado al intentar crear el Lakehouse en el área de trabajo '{identificador_ws}': {type(e).__name__} - {e}")
            
    return resultado

In [ ]:
payload = {
    "displayName": "DEV_datadaX",  # Nombre que se mostrará para el Lakehouse
    "type": "Lakehouse",               # Tipo de item a crear (obligatorio)
    "description": "Un Lakehouse de ejemplo creado con la API" # Opcional
}

print("--- Iniciando creación de Lakehouse ---")

respuesta = crear_lakehouse(payload=payload)

if respuesta:
    print("Detalles del Lakehouse creado:")
    print(f"  ID: {respuesta.get('id')}")
    print(f"  Nombre: {respuesta.get('displayName')}")
    print(f"  Tipo: {respuesta.get('type')}")
    print(f"  Descripción: {respuesta.get('description', 'N/A')}")
else:
    print("La creación del Lakehouse falló o no devolvió información.")